# Car Model Recognition Project
### Group 10: Tianyi Wei, Chong Chen, Jinzhan Li

## Project Introduction
### 1. Problem
The problem in hand is a car model recognition problem. By using the data set originated from Stanford University AI Lab, we hope to build a CNN model to tell the difference between different brands of cars. We believe our project is not only interesting for three big fans of car models, but also meaningful for businesses and the society. For example, by applying the model, manufacturers, including but not limited to car manufactures, are able to keep and improve their characteristics, and the police can detect the car model simply with a picture from monitoring system.  In brief, Fine-grained recognition is a growing subfield of computer vision that has many real-world applications. 
### 2. Dataset
The Cars dataset contains 16,185 images of 196 classes of cars. The data is split into 8,144 training images and 8,041 testing images, where each class has been split roughly in a 50-50 split. Classes are typically at the level of Make, Model, Year, ex. 2012 Tesla Model S or 2012 BMW M3 coupe.

Data source and banner image: http://ai.stanford.edu/~jkrause/cars/car_dataset.html contains all bounding boxes and labels for both training and tests. The dataset is officially publicized on kaggle: https://www.kaggle.com/jessicali9530/stanford-cars-dataset. 

**Citation:**
Krause, J., Stark, M., Deng, J., & Fei-Fei, L. (2013). 3d object representations for fine-grained categorization. In Proceedings of the IEEE International Conference on Computer Vision Workshops (pp. 554-561).
### 3. Methodology
CNN
### 4. Expected Outcome
Our goal is to build a neuron network model with an accuracy of 25%+ to correctly recognize the 196 classes of cars.
### 5. Milestone
Phase 1: Data processing, including reading data from matlab files, data cleaning and proprocessing, image preprocessing. Model frame constructing.

Phase 2: Model training and improving. 

Phase 3: Model testing and refining. Submit the outcome. 

### Here is the link, we use our model to predict the car model using randomly selected images from Carfax.
https://www.youtube.com/watch?v=rTYx9T65DqU

In [40]:
%cd '/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/'

/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset


reference below:

In [41]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html

In [42]:
#https://www.kaggle.com/eduardo4jesus/stanford-cars-dataset-a-quick-look-up

In [43]:
import scipy.io as sio
import pandas as pd
import numpy as np

In [44]:
car_label = sio.loadmat('cars_annos.mat',squeeze_me=True)
car_train=sio.loadmat('devkit/cars_train_annos.mat',squeeze_me=True)
car_test=sio.loadmat('devkit/cars_test_annos.mat',squeeze_me=True)

In [45]:
L=[]
for i in car_train['annotations']:
    if i[4] not in L:
        L.append(i[4])
L=sorted(L)
print(L[:5])

[1, 2, 3, 4, 5]


In [46]:
car_train['annotations'][5]

(259, 289, 515, 416, 123, '00006.jpg')

In [47]:
#process car_train['annotations']'s class, each -1, as it starts from 1 instead of 0(python default)
for record in car_train['annotations']:
        record[4]=record[4]-1

In [48]:
#processing car's label
#labels=[c for c in car_label['class_names']]
labels=[' '.join(c.split(' ')[:-2]) for c in car_label['class_names']]
labels=pd.DataFrame(labels,columns=['labels'])
labels

,labels
0,AM General Hummer
1,Acura RL
2,Acura TL
3,Acura TL
4,Acura TSX
5,Acura Integra Type
6,Acura ZDX
7,Aston Martin V8 Vantage
8,Aston Martin V8 Vantage
9,Aston Martin Virage


In [49]:
#Distinct car model
len(set([' '.join(c.split(' ')[:-2]) for c in car_label['class_names']]))

174

In [50]:
car_train['annotations'][10]

(51, 93, 601, 393, 48, '00011.jpg')

In [51]:
car_train['annotations']

array([(39, 116, 569, 375, 13, '00001.jpg'),
       (36, 116, 868, 587, 2, '00002.jpg'),
       (85, 109, 601, 381, 90, '00003.jpg'), ...,
       (26, 246, 660, 449, 162, '08142.jpg'),
       (78, 526, 1489, 908, 111, '08143.jpg'),
       (20, 240, 862, 677, 16, '08144.jpg')],
      dtype=[('bbox_x1', 'O'), ('bbox_y1', 'O'), ('bbox_x2', 'O'), ('bbox_y2', 'O'), ('class', 'O'), ('fname', 'O')])

In [52]:
dict={}
dict['filename']=[]
dict['class']=[]
dict['bbox_x1']=[]
dict['bbox_y1']=[]
dict['bbox_x2']=[]
dict['bbox_y2']=[]
for i in range((car_train['annotations'].shape[0])):
    dict['filename'].append(car_train['annotations'][i][5])
    dict['class'].append(car_train['annotations'][i][4])
    dict['bbox_x1'].append(car_train['annotations'][i][0])
    dict['bbox_y1'].append(car_train['annotations'][i][1])
    dict['bbox_x2'].append(car_train['annotations'][i][2])
    dict['bbox_y2'].append(car_train['annotations'][i][3])

In [53]:
train_df=pd.DataFrame(dict)
train_df.head()

,filename,class,bbox_x1,bbox_y1,bbox_x2,bbox_y2
0,00001.jpg,13,39,116,569,375
1,00002.jpg,2,36,116,868,587
2,00003.jpg,90,85,109,601,381
3,00004.jpg,133,621,393,1484,1096
4,00005.jpg,105,14,36,133,99


In [54]:
labels.head()

,labels
0,AM General Hummer
1,Acura RL
2,Acura TL
3,Acura TL
4,Acura TSX


In [55]:
train_df2=train_df.merge(labels,left_on='class',right_index=True)
train_df2.sort_values(by='filename',inplace=True)
train_df2.head(10)

,filename,class,bbox_x1,bbox_y1,bbox_x2,bbox_y2,labels
0,00001.jpg,13,39,116,569,375,Audi TTS
1,00002.jpg,2,36,116,868,587,Acura TL
2,00003.jpg,90,85,109,601,381,Dodge Dakota Club
3,00004.jpg,133,621,393,1484,1096,Hyundai Sonata Hybrid
4,00005.jpg,105,14,36,133,99,Ford F-450 Super Duty Crew
5,00006.jpg,122,259,289,515,416,Geo Metro
6,00007.jpg,88,88,80,541,397,Dodge Journey
7,00008.jpg,95,73,79,591,410,Dodge Charger
8,00009.jpg,166,20,126,1269,771,Mitsubishi Lancer
9,00010.jpg,57,21,110,623,367,Chevrolet Traverse


In [56]:
x=np.mean(train_df2['bbox_x2']-train_df2['bbox_x1'])
y=np.mean(train_df2['bbox_y2']-train_df2['bbox_y1'])
x,y

(573.3018172888015, 307.77038310412576)

In [57]:
# import pictures and resize pic's shape

In [58]:
import os

In [59]:
sorted(os.listdir('/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/cars_train'))[:10]

['00001.jpg',
 '00002.jpg',
 '00003.jpg',
 '00004.jpg',
 '00005.jpg',
 '00006.jpg',
 '00007.jpg',
 '00008.jpg',
 '00009.jpg',
 '00010.jpg']

In [60]:
import cv2

In [61]:
def preprocess_image(img): #resize pic to same size
    w, h = 224, 224 #
    img = cv2.resize(img,(w,h))
    img = img/255. #normalization 256种颜色组合 'RGB'

    return img

In [62]:
cv2.imread('/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/cars_train/00013.jpg').shape

(335, 500, 3)

In [63]:
preprocess_image(cv2.imread('/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/cars_train/00013.jpg')).shape

(224, 224, 3)

In [64]:
root='/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/cars_train'

In [65]:
#transform class using one hotcoding

In [66]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
data = array(train_df2['class'])
print(data)

[ 13   2  90 ... 162 111  16]


In [67]:
len(set(train_df2['class']))

196

In [68]:
# one hot encode
encoded_class = to_categorical(data) #transform all 8144 class using one hot coding by order
print(len(encoded_class))
encoded_class[30]

8144


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [69]:
int(train_df2.loc[train_df2['filename']=='00010.jpg'].index.values)

9

In [70]:
def image_generator(img_ids,batch_size=32):
    while True:
        onebatch=np.random.choice(img_ids,batch_size) #randomly choose 32 images make into one batch
        batch_input=[]
        batch_output=[]
        for img_id in onebatch: #img_id:'00010.jpg'
            input=preprocess_image(cv2.imread(root+'/'+img_id))
            output=encoded_class[int(train_df2.loc[train_df2['filename']==img_id].index.values)]
            #output=int(train_df2.loc[train_df2['filename']==img_id,'class'])
            batch_input+=[input]
            batch_output+=[output]
        batch_x=np.array(batch_input)
        batch_y=np.array(batch_output)
        
        yield (batch_x,batch_y)
            

In [71]:
train_img_ids=[x for x in train_df2['filename']]
np.random.shuffle(train_img_ids)

In [72]:
from sklearn.model_selection import train_test_split
train_img_ids, test_img_ids  = train_test_split(train_img_ids, test_size=0.25, random_state=42)
print(len(train_img_ids),len(test_img_ids))

6108 2036


In [73]:
train_img_ids[:10]

['00386.jpg',
 '07489.jpg',
 '04836.jpg',
 '06603.jpg',
 '04941.jpg',
 '06815.jpg',
 '01995.jpg',
 '01421.jpg',
 '03093.jpg',
 '06789.jpg']

In [74]:
train_genrator=image_generator(train_img_ids)
test_generator=image_generator(test_img_ids)

In [75]:
#neural network

In [76]:
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau

In [77]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(224,224,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(196, activation='softmax', name='predictions')(x)

#Create your own model 
model = Model(input=input, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

/Users/wty24/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`


In [78]:
sgd = optimizers.SGD(lr=0.001, decay=0, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['categorical_accuracy'])

In [79]:
train_steps=len(train_img_ids)//32
train_steps

190

In [46]:
model.fit_generator(train_genrator,epochs=8,steps_per_epoch=train_steps)

Epoch 1/8
190/190 [==============================] - 6256s 33s/step - loss: 5.1982 - categorical_accuracy: 0.0230
Epoch 2/8
190/190 [==============================] - 6626s 35s/step - loss: 4.2740 - categorical_accuracy: 0.1434
Epoch 3/8
190/190 [==============================] - 5921s 31s/step - loss: 2.8056 - categorical_accuracy: 0.3924
Epoch 4/8
190/190 [==============================] - 5451s 29s/step - loss: 1.4040 - categorical_accuracy: 0.6794
Epoch 5/8
190/190 [==============================] - 5496s 29s/step - loss: 0.6534 - categorical_accuracy: 0.8482
Epoch 6/8
190/190 [==============================] - 5483s 29s/step - loss: 0.3024 - categorical_accuracy: 0.9293
Epoch 7/8
190/190 [==============================] - 5498s 29s/step - loss: 0.1597 - categorical_accuracy: 0.9660
Epoch 8/8
190/190 [==============================] - 5744s 30s/step - loss: 0.1128 - categorical_accuracy: 0.9791


In [80]:
from keras.models import load_model
model.save('vgg16_CarModel_Recognition)')

In [49]:
model.evaluate_generator(test_generator,steps=len(test_img_ids)//32)[1]

0.2986111111111111

### Our model accuracy is 29.9%

In [85]:
# baseline 
a=[int(train_df2.loc[train_df2['filename']==test,'class']) for test in test_img_ids]
print(labels.loc[pd.Series(a).value_counts()[0]],pd.Series(a).value_counts()[0]/len(a))

labels    Audi R8
Name: 14, dtype: object 0.0068762278978389


### Our baseline accuracy is 0.6%. 

In [109]:
model.load_weights('vgg16_CarModel_Recognition)')

In [110]:
#use model to predict

In [111]:
dict2={}
dict2['filename']=[]
dict2['bbox_x1']=[]
dict2['bbox_y1']=[]
dict2['bbox_x2']=[]
dict2['bbox_y2']=[]
for i in range((car_test['annotations'].shape[0])):
    dict2['filename'].append(car_test['annotations'][i][4])
    dict2['bbox_x1'].append(car_test['annotations'][i][0])
    dict2['bbox_y1'].append(car_test['annotations'][i][1])
    dict2['bbox_x2'].append(car_test['annotations'][i][2])
    dict2['bbox_y2'].append(car_test['annotations'][i][3])
test_df=pd.DataFrame(dict2)
test_df.head()

,filename,bbox_x1,bbox_y1,bbox_x2,bbox_y2
0,00001.jpg,30,52,246,147
1,00002.jpg,100,19,576,203
2,00003.jpg,51,105,968,659
3,00004.jpg,67,84,581,407
4,00005.jpg,140,151,593,339


In [112]:
testroot='/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/cars_test'

In [113]:
def testimage_generator(img_ids,batch_size=32):
    while True:
        onebatch=np.random.choice(img_ids,batch_size) #randomly choose 32 images make into one batch
        batch_input=[]
        batch_output=[]
        for img_id in onebatch: #img_id:'00010.jpg'
            input=preprocess_image(cv2.imread(testroot+'/'+img_id))
            output=encoded_class[int(train_df2.loc[train_df2['filename']==img_id].index.values)]
            #output=int(train_df2.loc[train_df2['filename']==img_id,'class'])
            batch_input+=[input]
            batch_output+=[output]
        batch_x=np.array(batch_input)
        batch_y=np.array(batch_output)
        
        yield (batch_x,batch_y)

In [114]:
test=sorted(os.listdir('/Users/wty24/Desktop/2019SpringTerm/758B/Project/stanford-cars-dataset/cars_test/'))[:80]
len(test)

80

In [115]:
def return_class(list):
    return labels.loc[np.argmax(list)]


In [116]:
return_class(model.predict(np.array([preprocess_image(cv2.imread(testroot+'/'+'03268.jpg'))])))

labels    Dodge Ram Pickup 3500 Quad
Name: 86, dtype: object

In [117]:
from PIL import Image
import urllib.request

URL = 'https://carfax-img.vast.com/carfax/-1878584670847682377/1/344x258'
with urllib.request.urlopen(URL) as url:
    with open('temp.jpg', 'wb') as f:
        f.write(url.read())

img = Image.open('temp.jpg')

img.show()

In [118]:
return_class(model.predict((np.array([preprocess_image(cv2.imread('temp.jpg'))]))))

labels    Bentley Continental Supersports Conv.
Name: 38, dtype: object